#### Decision Tree, Ensemble 모델
231019

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score
# import warnings
# warnings.filterwarnings("ignore")
from tqdm import tqdm

# K-Fold
from sklearn.model_selection import KFold
kf=KFold(n_splits=5, shuffle=True)

#### Random Forest, XGBoost, GrandientBoost 3개의 모델 생성

In [2]:
RF=RandomForestClassifier(max_depth=20, n_estimators=200, random_state=0)
XGB= XGBClassifier(n_estimators=200, max_depth=4,n_jobs=-1)
GBM = GradientBoostingClassifier(max_depth=3, n_estimators=200, random_state=0)

#### score 저장용 dict

In [3]:
score = {
    'RF' :{
        'accuracy' : [], 'precision' : [], 'recall' : [], 'f1_score' : [],
    },
    'XGB' :{
        'accuracy' : [], 'precision' : [], 'recall' : [], 'f1_score' : [],
    },
    'GBM' :{
        'accuracy' : [], 'precision' : [], 'recall' : [], 'f1_score' : [],
    },
    'DNN' :{
        'accuracy' : [], 'precision' : [], 'recall' : [], 'f1_score' : [],
    },
}

#### 일단 MinMax Scaling

In [4]:
import pandas as pd

origin_csv = pd.read_csv('data/Glucose_Set-2A-1.csv')
# df = df.sample(frac = 1)
usable_data = pd.DataFrame()
usable_data = pd.concat([origin_csv['mPD1_AL-D'], origin_csv['mPD2_AL-D'], origin_csv['mPD3_AL-D'], 
                         origin_csv['T-rPD_AL-D'], origin_csv['R-rPD_AL-D'],
                         origin_csv['LD_C_Av'], origin_csv['T-rPD'], origin_csv['Th3_C'], origin_csv['Th4_C'], origin_csv['Th6_C'],
                         origin_csv['Th3_AvD'], origin_csv['Th4_AvD'], origin_csv['Th6_AvD'],
                         origin_csv['mPD1'], origin_csv['mPD2'], origin_csv['mPD3'], origin_csv['Glucose']], axis = 1, sort = False)

usable_data = usable_data.sample(frac = 1)

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

mMscaler = MinMaxScaler()
mMscaler.fit(usable_data)

mMscaled_data = mMscaler.transform(usable_data)
x_data = pd.DataFrame(mMscaled_data)

x_data = x_data.replace(('Glucose',1.0), 8)
x_data = x_data.replace(('Glucose',0.3), 1)
x_data = x_data.replace(('Glucose',0.4), 2)
x_data = x_data.replace(('Glucose',0.5), 3)
x_data = x_data.replace(('Glucose',0.6), 4)
x_data = x_data.replace(('Glucose',0.7000000000000001), 5)
x_data = x_data.replace(('Glucose',0.8), 6)
x_data = x_data.replace(('Glucose',0.9), 7)
# x_data = x_data.replace(('Glucose',0.7), 5)

In [5]:
y_input = x_data[16]

x_input = x_data[0]
x_input = pd.concat([x_input ,x_data[1]], axis = 1, sort = False) 
x_input = pd.concat([x_input ,x_data[2]], axis = 1, sort = False)
x_input = pd.concat([x_input ,x_data[3]], axis = 1, sort = False)
x_input = pd.concat([x_input ,x_data[4]], axis = 1, sort = False)
x_input = pd.concat([x_input ,x_data[5]], axis = 1, sort = False)
x_input = pd.concat([x_input ,x_data[6]], axis = 1, sort = False)
x_input = pd.concat([x_input ,x_data[7]], axis = 1, sort = False)
x_input = pd.concat([x_input ,x_data[8]], axis = 1, sort = False)
x_input = pd.concat([x_input ,x_data[9]], axis = 1, sort = False)
x_input = pd.concat([x_input ,x_data[10]], axis = 1, sort = False)
x_input = pd.concat([x_input ,x_data[11]], axis = 1, sort = False)
x_input = pd.concat([x_input ,x_data[12]], axis = 1, sort = False)
x_input = pd.concat([x_input ,x_data[13]], axis = 1, sort = False)
x_input = pd.concat([x_input ,x_data[14]], axis = 1, sort = False)
x_input = pd.concat([x_input ,x_data[15]], axis = 1, sort = False)

# df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(df_x_data, df_y_data, test_size=0.3, random_s
df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(x_input, y_input, test_size=0.1, random_state=777, stratify=y_input)

#### 3개의 모델에 대한 train

In [6]:
# RF.fit(x_input, y_input.ravel())
# XGB.fit(x_input, y_input.ravel(), eval_metric=["logloss"], verbose=True)
# GBM.fit(x_input, y_input.ravel())

In [7]:
for train_index, test_index in tqdm(kf.split(df_x_train, df_y_train)):
    x_train, x_test, y_train, y_test=\
    df_x_train.iloc[train_index], df_x_train.iloc[test_index], df_y_train.iloc[train_index], df_y_train.iloc[test_index]
    
    RF.fit(x_train, y_train.ravel())
    XGB.fit(x_train, y_train.ravel(), eval_metric=["logloss"], verbose=True)
    GBM.fit(x_train, y_train.ravel())
#     DNN.fit(x_train, encoder.fit_transform(y_train), batch_size=10, epochs=50)
    
    score['RF']['accuracy'].append(RF.score(x_test, y_test.values.ravel()))
    
    score['XGB']['accuracy'].append(XGB.score(x_test, y_test.values.ravel()))
    
    score['GBM']['accuracy'].append(GBM.score(x_test, y_test.values.ravel()))


# score['RF']['accuracy'].append(RF.score(df_x_train, df_y_train.ravel()))
# # score['RF']['precision'].append(precision_score(RF.predict(df_x_train),  df_y_train))
# # score['RF']['recall'].append(recall_score(RF.predict(df_x_train),  df_y_train))
# # score['RF']['f1_score'].append(f1_score(RF.predict(df_x_train),  df_y_train))

# score['XGB']['accuracy'].append(XGB.score(df_x_train, df_y_train.values.ravel()))
# # score['XGB']['precision'].append(precision_score(XGB.predict(df_x_train), df_y_train))
# # score['XGB']['recall'].append(recall_score(XGB.predict(df_x_train), df_y_train))
# # score['XGB']['f1_score'].append(f1_score(XGB.predict(df_x_train), df_y_train))

# score['GBM']['accuracy'].append(GBM.score(df_x_train, df_y_train.values.ravel()))
# # score['GBM']['precision'].append(precision_score(GBM.predict(df_x_train), df_y_train))
# # score['GBM']['recall'].append(recall_score(GBM.predict(df_x_train), df_y_train))
# # score['GBM']['f1_score'].append(f1_score(GBM.predict(df_x_train), df_y_train))

0it [00:00, ?it/s]C:\Users\ojski\anaconda3\envs\HnsEnv\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
1it [00:28, 28.62s/it]C:\Users\ojski\anaconda3\envs\HnsEnv\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
2it [00:56, 28.38s/it]C:\Users\ojski\anaconda3\envs\HnsEnv\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
3it [01:25, 28.44s/it]C:\Users\ojski\anaconda3\envs\HnsEnv\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compati

In [8]:
import numpy as np
print(f'1-1. RandomForestClassifier [평균 Accuracy] :', np.mean(score['RF']['accuracy'])*100 , ',  [Accuracy 편차] :', np.std(score['RF']['accuracy']))

print(f'2-1. XGBOOSTClassifier [평균 Accuracy] :', np.mean(score['XGB']['accuracy'])*100 , ',  [Accuracy 편차] :', np.std(score['XGB']['accuracy']))

print(f'3-1. GradientBoostingClassifier [평균 Accuracy] :', np.mean(score['GBM']['accuracy'])*100 , ',  [Accuracy 편차] :', np.std(score['GBM']['accuracy']))

1-1. RandomForestClassifier [평균 Accuracy] : 99.44181737468878 ,  [Accuracy 편차] : 0.003899938840550474
2-1. XGBOOSTClassifier [평균 Accuracy] : 98.57913055786094 ,  [Accuracy 편차] : 0.00629278054593741
3-1. GradientBoostingClassifier [평균 Accuracy] : 97.79306839602914 ,  [Accuracy 편차] : 0.0026048218262329026


In [19]:
print(df_x_test[0:3])

            0         1         2         3         4         5         6   \
3436  0.997971  0.000199  0.000171  0.997134  0.773587  0.097087  0.997060   
1682  0.440758  0.000155  0.000403  0.375299  0.371071  0.000000  0.375200   
3714  0.000613  0.000334  0.531098  0.000281  0.122640  0.699029  0.000401   

            7         8         9         10        11        12        13  \
3436  0.050331  0.020000  0.747875  0.012179  0.002257  0.743565  0.997546   
1682  0.598675  0.393333  0.648725  0.607578  0.392777  0.651096  0.440370   
3714  0.218543  0.320000  0.880076  0.216509  0.316027  0.882745  0.000578   

            14        15  
3436  0.000198  0.000246  
1682  0.000296  0.000615  
3714  0.000395  0.531358  


In [16]:
result_rf = RF.predict(df_x_test[0:10])
result_xgb = XGB.predict(df_x_test[0:10])
result_gbm = GBM.predict(df_x_test[0:10])
print(result_rf[0:10])
print(result_xgb[0:10])
print(result_gbm[0:10])
print(df_y_test[0:10])

[3. 3. 7. 8. 0. 1. 2. 5. 8. 3.]
[3 3 7 8 0 1 2 5 8 3]
[3. 3. 7. 8. 0. 1. 2. 5. 8. 3.]
3436    3.0
1682    3.0
3714    7.0
686     8.0
2061    0.0
447     1.0
1927    2.0
2985    5.0
3449    8.0
2820    3.0
Name: 16, dtype: float64


In [11]:
import joblib

XGB.save_model("property/xgb_test.json")
joblib.dump(RF31, "property/rf_test.joblib")
joblib.dump(GB31, "property/gb_test.joblib")

['property/gbm_test.joblib']